In [1]:
#importing required libraries
import numpy as np
import itertools
import matplotlib.pyplot as plt
from pathlib import Path
import os

import re
import matplotlib.pyplot as plt


In [2]:
class reducing_gates(object):

    def __init__(self, n_q, n_g, seq_ID, CNOT_seq):

        self.n_q = n_q                          # number of qubits
        self.n_g = n_g                          # number of gates
        self.seq_ID = seq_ID
        #self.CNOT_seq = [[[3, 1]], [[0, 3]], [[2, 0]], [[2, 3]], [[1, 3]], [[1, 0]], [[3, 2]], [[0, 2]]]
        self.CNOT_seq =  [[0,0]]*n_g            # initial random CNOT sequence
        self.tmat_i = np.identity(n_q)          # initial transformation matrix
        self.tmat_f = self.tmat_i               # final transformation matrix which becomes identity
        self.red_CNOT_seq = []                  # list storing reduced CNOT operations

        print('Check: \n Number of qubits:',self.n_q,'\n Number of gates:',self.n_g,'\n')

    # function to generate random sequence of n_g CNOT gates among n_q qubits
    def rg_CNOT_seq(self):

        all_possible_cnots = list(itertools.permutations(range(self.n_q),2))
        selected_cnots = np.random.choice(len(all_possible_cnots),self.n_g,replace=True)
        self.CNOT_seq = [all_possible_cnots[i] for i in selected_cnots]

        #for i in range(0,self.n_g):
        #    self.CNOT_seq[i] = np.random.choice(np.arange(0,self.n_q).tolist(),(1,2),replace=False)

        return self.CNOT_seq

    # function to generate transformation matrix from the sequence of CNOT gates
    def rg_gen_tmat(self):

        for i in range(0,len(self.CNOT_seq)):
            #print(self.CNOT_seq)
            c = self.CNOT_seq[i][0]
            t = self.CNOT_seq[i][1]

            for j in range(0,self.n_q):
                self.tmat_i[t][j] = int(self.tmat_i[c][j]) ^ int(self.tmat_i[t][j])

            self.tmat_f = self.tmat_i
            #self.tmat_i = np.identity(self.n_q)
        return self.tmat_f

    # function to perform XOR operation between two rows
    def rg_add_rows(self, rowc, rowt):

        for i in range(0,self.n_q):
          self.tmat_f[rowt][i] = int(self.tmat_f[rowc][i]) ^ int(self.tmat_f[rowt][i])

        return

    # function to check rows
    def rg_check_rows(self, rowi):

        for j in range(rowi,self.n_q):
            if self.tmat_f[j][rowi] == 1:
                for k in range(j+1,self.n_q):
                    if self.tmat_f[k][rowi] == 1:
                        self.red_CNOT_seq.append([j,k])
                        print(j,k,'row \n')
                        self.rg_add_rows(j,k)
                        break
        return

    # function to check columns
    def rg_check_cols(self, coli):

        for j in range(coli,-1,-1):
            if self.tmat_f[j][coli] == 1:
                for k in range(j-1,-1,-1):
                    if self.tmat_f[k][coli] == 1:
                        self.red_CNOT_seq.append([j,k])
                        print(j,k,'col \n')
                        self.rg_add_rows(j,k)
                        break
        return

    # optimizing algoritm to reduce number of gates
    def rg_optimize(self):

        for i in range(0,self.n_q):
            if self.tmat_f[i][i] == 0:
                try:
                    jn = self.tmat_f[:,i].tolist().index(1,i+1,self.n_q)
                except:
                    print('skipped',i,'\n')
                    continue
                else:
                    print(jn,i,'added',i,'\n')
                    self.red_CNOT_seq.append([jn,i])
                    self.rg_add_rows(jn,i)

            self.rg_check_rows(i)
            sumr = 0
            for p in range(i,self.n_q):
                sumr = sumr + self.tmat_f[p][i]
            while sumr > 1:
                print(i,'rep row \n')
                self.rg_check_rows(i)
                sumr = 0
                for p in range(i,self.n_q):
                    sumr = sumr + self.tmat_f[p][i]

        for i in range(self.n_q-1,-1,-1):
            self.rg_check_cols(i)
            sumc = 0
            for q in range(i,-1,-1):
                sumc = sumc + self.tmat_f[q][i]
            while sumc > 1:
                print(i,'rep col \n')
                self.rg_check_cols(i)
                sumc = 0
                for q in range(i,-1,-1):
                    sumc = sumc + self.tmat_f[q][i]

        return self.tmat_f, self.red_CNOT_seq

    def write_CNOTs(self,path,gates):
        Path(path).mkdir(parents=True, exist_ok=True)
        with open(path + '/circuit_' + self.seq_ID + '.txt', 'w+') as f:
                f.write(str(self.n_q) + '\n')
                for gate in gates:
                    f.write('CNOT ' + str(gate[0]) + ' ' + str(gate[1]) + '\n')
        return


In [3]:
def load_random_cnots_dynamic(file):
        with open(file, 'r') as f:
            data = f.read()
        list_of_initial_cnots = [[int(cnot.split()[0]),int(cnot.split()[1])] for cnot in re.findall('\d{1,2} \d{1,2}',data)]
        return list_of_initial_cnots

def compute_t_layer_displacement_and_cnots(gates):
    displacement = 0
    displacement_list = []
    cnots = 0
    selected = [False]*len(gates)
    for idx1,gate1 in enumerate(gates):
        displacement = 0
        cnots = 0
        if selected[idx1]==True:
            continue
        cnots += 3
        displacement += (1+abs(gate1[1]-gate1[0]))
        cnots += 1
        displacement_list.append([gate1,displacement,cnots])
        #print(gate1,displacement,cnots)
        prev = gate1[1]
        selected[idx1]=True
        for idx2,gate2 in enumerate(gates[idx1+1:]):
            if selected[idx1+idx2+1]==True:
                break
            if gate2[0]==gate1[0]:
                selected[idx1+idx2+1]=True
                cnots += 1
                displacement += abs(gate2[1]-prev)
                displacement_list.append([gate2,displacement,cnots])
                #print(gate2,displacement,cnots)
                prev = gate2[1]
            else:
                break
        displacement+= (1+abs(gate1[0]-prev))
        cnots += 3
        displacement_list.append([displacement,cnots])
        #print(displacement,cnots)
    return displacement_list

In [ ]:
#TO GENERATE INITIAL CNOT CIRCUTS

num_qubits = 32
list_of_cnots = range(5,260,5)
num_realizations = 100

initial_num_cnots = []
compiled_num_cnots = []

for num_cnots in list_of_cnots:
    num_cnot_initial = []
    num_cnot_compiled = []
    for i in range(num_realizations):
        circ = reducing_gates(num_qubits, num_cnots, str(num_cnots) + '_' + str(i),[])
        initial_cnots = circ.rg_CNOT_seq()
        circ.write_CNOTs(f'D:\\IITBombay\\BTP\\trial\\qubit32\\i_circuits',initial_cnots)
        trans_mat = circ.rg_gen_tmat()
        final_trans_mat, compiled_cnots = circ.rg_optimize()
        circ.write_CNOTs(f'D:\\IITBombay\\BTP\\trial\\qubit32\\c_circuits',compiled_cnots)

In [ ]:
num_initial_cnots = []
num_initial_cnots_physical = []

num_compiled_cnots = []
num_compiled_cnots_physical = []

num_static_cnots = []

#create a list which goes as 5,10,...,255
num_qubits = 32
list_of_cnots = range(5,260,5)
num_realizations = 100

for num_cnots in list_of_cnots:
    initial_cnots_num = []
    compiled_cnots_num = []
    static_cnots_num = []

    for i in range(num_realizations):
        path = f'D:\\IITBombay\\BTP\\trial\\qubit32\\i_circuits\\circuit_'+str(num_cnots)+'_'+str(i)+'.txt'
        with open(path,'r') as f:
            data = f.read()
        initial_cnots_num.append(len(re.findall('\d{1,2} \d{1,2}',data)))

        path = f'D:\\IITBombay\\BTP\\trial\\qubit32\\c_circuits\\circuit_'+str(num_cnots)+'_'+str(i)+'.txt'
        with open(path,'r') as f:
            data = f.read()
        compiled_cnots_num.append(len(re.findall('\d{1,2} \d{1,2}',data)))

        path = f'D:\\IITBombay\\BTP\\trial\\qubit32\\stat_circuits\\circuit_'+str(num_cnots)+'_'+str(i)+'_static.txt'
        with open(path,'r') as f:
            data = f.readlines()[1]
        static_cnots_num.append(int(re.findall(r'\d+', data)[0]))

    num_initial_cnots.append(initial_cnots_num)
    num_compiled_cnots.append(compiled_cnots_num)
    num_static_cnots.append(static_cnots_num)

num_initial_cnots = np.array(num_initial_cnots)
num_compiled_cnots = np.array(num_compiled_cnots)
num_static_cnots = np.array(num_static_cnots)


In [ ]:
num_moves_initial = []
num_moves_compiled = []

#create a list which goes as 5,10,...,255
num_qubits = 32
list_of_cnots = range(5,260,5)
num_realizations = 100

for num_cnots in list_of_cnots:

    initial_moves_num = []
    compiled_moves_num = []

    for i in range(num_realizations):

        path = f'D:\\IITBombay\\BTP\\trial\\qubit32\\i_circuits\\circuit_'+str(num_cnots)+'_'+str(i)+'.txt'
        initial_file = load_random_cnots_dynamic(path)
        initial_disps = compute_t_layer_displacement_and_cnots(initial_file)
        n_init = 0.
        for j in initial_disps:
            if len(j)==2: n_init += j[0]
        initial_moves_num.append(n_init)

        path = f'D:\\IITBombay\\BTP\\trial\\qubit32\\c_circuits\\circuit_'+str(num_cnots)+'_'+str(i)+'.txt'
        compiled_file = load_random_cnots_dynamic(path)
        compiled_disps = compute_t_layer_displacement_and_cnots(compiled_file)
        n_comp = 0.
        for j in compiled_disps:
            if len(j)==2: n_comp += j[0]
        compiled_moves_num.append(n_comp)

    num_moves_initial.append(initial_moves_num)
    num_moves_compiled.append(compiled_moves_num)

num_moves_initial = np.array(num_moves_initial)
num_moves_compiled = np.array(num_moves_compiled)

In [ ]:
plt.errorbar(num_initial_cnots[:,-1], num_moves_initial.mean(axis=1), yerr = num_moves_initial.std(axis=1), label = 'static', fmt='o--')
plt.errorbar(num_initial_cnots[:,-1], num_moves_compiled.mean(axis=1), yerr = num_moves_compiled.std(axis=1), label = 'compiled dynamic', fmt='o--')
plt.xlabel('Sampled number of CNOTs')
plt.ylabel('Compiled number of CNOTs')
plt.legend()
plt.show()

In [ ]:
plt.errorbar(num_initial_cnots[:,-1], num_static_cnots.mean(axis=1), yerr = num_static_cnots.std(axis=1), label = 'static', fmt='o--')
plt.errorbar(num_initial_cnots[:,-1], num_compiled_cnots.mean(axis=1), yerr = num_compiled_cnots.std(axis=1), label = 'compiled dynamic', fmt='o--')
plt.xlabel('Sampled number of CNOTs')
plt.ylabel('Number of atom moves')
plt.legend()
plt.show()